In [166]:
import numpy as np
chardict = {"A":0,"B":1,"C":2,"D":3}

In [167]:
strs1 = ['AABBCCDD', 'ABBCBBDD', 'ACBCBCD', 'AD', 'ACBCBABCDD',
         'BABAADDD', 'BABCDCC', 'ABDBBCCDD', 'ABAAACDCCD', 'ABD']

In [168]:
strs2 = ['DDCCBBAA', 'DDABCBA', 'CDCDCBABA', 'DDBBA', 'DADACBBAA',
         'CDDCCBA', 'BDDBCAAAA', 'BBABBDDDCD', 'DDADDBCAA', 'DDCAAA']

In [169]:
def parameter_estimation_per_string(s, transition, observation, pie):
## Expectation 
    alpha = np.zeros((3, len(s)))
    beta = np.zeros((3, len(s)))
    gamma = np.zeros((3, len(s)))
    zeta = np.zeros((3, 3, len(s)))
    pie1 = np.zeros(3)
    observation1 = np.zeros((3, 4))
    transition1 = np.zeros((3, 3))

    ## computing alpha
    for t, c in enumerate(s):
        for i in range(0,3):
            if t == 0:
                alpha[i,0] = pie[i]*observation[i, chardict[s[0]]]
            else:
                total_sum = 0
                for j in range(0,3):
                    total_sum += (alpha[j,t-1] * transition[j,i])
                alpha[i,t] = observation[i,chardict[c]] * total_sum

    # computing beta
    T = len(s)-1
    for x, c in enumerate(s):
        for i in range(0,3):
            t = T-x
            if t == T:
                beta[i, t] = 1
            else:
                total_sum = 0
                for j in range(0,3):
                    total_sum += (beta[j,t+1] * transition[i,j] 
                                  * observation[j, chardict[s[t+1]]])
                beta[i, t] = total_sum
                
    prob_s = 0
    T = len(s)-1    
    for i in range(0,3):
        prob_s += alpha[i, T]

    #print("######Gamma###########")
    for t, c in enumerate(s):
        for i in range(0, 3):
            gamma[i, t] = (alpha[i, t] * beta[i, t]) / prob_s
            #print(gamma[i, t], end= " ")
        #print()

    #print("######Zeta###########")

    for t, c in enumerate(s):
        if(t!=len(s)-1):
            for i in range(0, 3):
                for j in range(0, 3):
                    zeta[i, j, t] = ((alpha[i, t] * transition[i, j]) 
                                     * (beta[j, t+1] * observation[j, chardict[s[t+1]]])) / prob_s

    #print("#####Updated start probabilities")
        #print(pie1[i], end=" ")
    #print()
    
    zetaOutNum = np.zeros((3,3))
    gammaOutTransDenom = np.zeros(3)
    gammaOutObsDenom = np.zeros(3)
    gammaOutObsNum = np.zeros((3,4))

    #print("###########Updated transistion matrix")
    
    for i in range(0, 3):
        for t, c in enumerate(s):
             if t!=T:
                gammaOutTransDenom[i] += gamma[i, t]
    
    for i in range(0, 3):
        for j in range(0, 3):
            for t, c in enumerate(s):
                if t!=T:
                    zetaOutNum[i,j] += zeta[i, j, t]
                    
    for i in range(0, 3):
        for t, c in enumerate(s):
            gammaOutObsDenom[i] += gamma[i, t]
                    
    for i in range(0,3):
        for t,c in enumerate(s):
                gammaOutObsNum[i, chardict[c]] += gamma[i, t]
                
    for i in range(0,3):
        pie1[i] = gamma[i, 0] / prob_s
    
    return zetaOutNum, gammaOutTransDenom, gammaOutObsNum, gammaOutObsDenom, pie1, 1/prob_s

    

In [178]:
transition_w1 = np.matrix([[0.3, 0.4, 0.4], 
               [0.4, 0.2, 0.4],
               [0.5, 0.3, 0.2]])
observation_w1 = np.matrix([[0.4, 0.1, 0.4, 0.1], 
               [0.1, 0.3, 0.4, 0.2],
               [0.4, 0.2, 0.2, 0.2]])
pie_w1 = np.array([0.33, 0.33, 0.33])
prev_transition_w1 = np.zeros((3, 3))
prev_observation_w1 = np.zeros((3, 4))
prev_pie_w1 = np.zeros(3)
threshold = 0.001

n = 1
while True:
    
    prev_transition_w1 = np.copy(transition_w1)
    prev_observation_w1 = np.copy(observation_w1)
    prev_pie_w1 = np.copy(pie_w1)
    
    trans_num = np.zeros((3,3))
    trans_denom = np.zeros(3)
    obs_num = np.zeros((3, 4))
    obs_denom = np.zeros(3)
    pie_num = np.zeros(3)
    pie_denom = 0

    for s in strs1:
        trans_num_temp, trans_denom_temp, obs_num_temp, obs_denom_temp, pie_num_temp, pie_denom_temp = parameter_estimation_per_string(s, transition_w1, observation_w1, pie_w1)
        trans_num = np.add(trans_num, trans_num_temp)
        trans_denom = np.add(trans_denom, trans_denom_temp)
        obs_num = np.add(obs_num, obs_num_temp)
        obs_denom = np.add(obs_denom, obs_denom_temp)
        pie_num = np.add(pie_num, pie_num_temp)
        pie_denom += pie_denom_temp

    for i in range(0, 3):
        for j in range(0, 3):
            transition_w1[i, j] = trans_num[i,j] / trans_denom[i]
            
    count = 0
    
    for i in range(0, 3):
        for j in range(0, 3):
            if abs(transition_w1[i,j] - prev_transition_w1[i, j]) <= threshold:
                count = count+1
        

    for i in range(0, 3):
        for j in range(0, 4):
            observation_w1[i, j] = obs_num[i,j] / obs_denom[i]
            
    for i in range(0, 3):
        pie_w1[i] =  pie_num[i] / pie_denom
            
    if(count/9 >= 0.95):
        break
    n += 1
    
print("Total Iterations : ", n)
print("#####Transition matrix######")
for i in range(0,3):
    for j in range(0,3):
        print(transition_w1[i,j], end=" ")
    print()
    
print("#####Observation matrix######")
for i in range(0,3):
    for j in range(0,4):
        print(observation_w1[i,j], end=" ")
    print()


Total Iterations :  34
#####Transition matrix######
0.3751147707145016 0.3328520763356431 0.29203315294985527 
1.1478609137498578e-12 0.9098581257271908 0.09014187427166127 
0.31417818185361934 0.6828005241779244 0.0030212939684563338 
#####Observation matrix######
0.8290323708475977 0.17096718439551029 4.4475682101026104e-07 7.105119696295795e-14 
3.246799242270759e-14 0.17745906913302117 0.3995190002194299 0.4230219306475166 
0.0003759151752028186 0.9995097919756247 5.917635230979648e-05 5.5116496862800854e-05 


In [179]:
transition_w2 = np.matrix([[0.4, 0.2, 0.4], 
               [0.7, 0.2, 0.1],
               [0.5, 0.3, 0.2]])
observation_w2 = np.matrix([[0.1, 0.1, 0.4, 0.4], 
               [0.1, 0.1, 0.4, 0.4],
               [0.1, 0.2, 0.4, 0.3]])
pie_w2 = np.array([0.33, 0.33, 0.33])

n = 1
while True:
    trans_num = np.zeros((3,3))
    trans_denom = np.zeros(3)
    obs_num = np.zeros((3, 4))
    obs_denom = np.zeros(3)
    
    prev_transition_w2 = np.copy(transition_w2)
    prev_observation_w2 = np.copy(observation_w2)
    prev_pie_w2 = np.copy(pie_w2)
    pie_num = np.zeros(3)
    pie_denom = 0
    
    for s in strs2:
        trans_num_temp, trans_denom_temp, obs_num_temp, obs_denom_temp, pie_num_temp, pie_denom_temp = 
        parameter_estimation_per_string(s, transition_w2, observation_w2, pie_w2)
        trans_num = np.add(trans_num, trans_num_temp)
        trans_denom = np.add(trans_denom, trans_denom_temp)
        obs_num = np.add(obs_num, obs_num_temp)
        obs_denom = np.add(obs_denom, obs_denom_temp)
        pie_num = np.add(pie_num, pie_num_temp)
        pie_denom += pie_denom_temp
        
    for i in range(0, 3):
        for j in range(0, 3):
            transition_w2[i, j] = trans_num[i,j] / trans_denom[i]
    for i in range(0, 3):
        for j in range(0, 4):
            observation_w2[i, j] = obs_num[i,j] / obs_denom[i]
            
    for i in range(0, 3):
        pie_w2[i] =  pie_num[i] / pie_denom
            
    count = 0
    
    for i in range(0, 3):
        for j in range(0, 3):
            if abs(transition_w2[i,j] - prev_transition_w2[i, j]) <= threshold:
                count = count+1
    if(count/9 >= 0.95):
        break
    n += 1
    
print("Total Iterations : ", n)
print("#####Transition matrix######")
for i in range(0,3):
    for j in range(0,3):
        print(transition_w2[i,j], end=" ")
    print()
    
print("#####Observation matrix######")
for i in range(0,3):
    for j in range(0,4):
        print(observation_w2[i,j], end=" ")
    print()


Total Iterations :  49
#####Transition matrix######
0.9133352005005944 1.7170383427527758e-08 0.08666478232902212 
0.6551916619471053 0.20696464389972788 0.13784369415316688 
1.1814011007062984e-14 0.9935737644236262 0.0064262355763618885 
#####Observation matrix######
0.43738736516911203 0.36452728108157345 0.1980853537456902 3.6242029940031485e-12 
0.08636460572533686 4.823445876659286e-16 0.06319560352393361 0.850439790750729 
1.0579213044052854e-09 1.7837037443862464e-07 0.23853238485628928 0.7614674357154149 


In [180]:
###################### 2nd part ######################

def probability_of_a_sequence(s1, trans, obs, p):
    alpha = np.zeros((3, len(s1)))
    for t, c in enumerate(s1):
        for i in range(0,3):
            if t == 0:
                alpha[i,0] = p[i]*obs[i, chardict[s1[0]]]
            else:
                total_sum = 0
                for j in range(0,3):
                    total_sum += (alpha[j,t-1] * trans[j,i])
                alpha[i,t] = obs[i,chardict[c]] * total_sum
                
    
    total_prob = 0
    T = len(s1)-1
    for i in range(0,3):
        total_prob += alpha[i, T]
    return total_prob


In [181]:
p_w1 = 0.5
p_w2 = 0.5
        
string1 = 'ABBBCDDD'
prob_ABBBCDDD_given_w1 = probability_of_a_sequence(string1, transition_w1, observation_w1, pie_w1)
prob_ABBBCDDD_given_w2 = probability_of_a_sequence(string1, transition_w2, observation_w2, pie_w2)
prob_w1_given_ABBBCDDD = prob_ABBBCDDD_given_w1 * p_w1
prob_w2_given_ABBBCDDD = prob_ABBBCDDD_given_w2 * p_w2

print("prob_w1_given_ABBBCDDD = ", prob_w1_given_ABBBCDDD)
print("prob_w2_given_ABBBCDDD = ", prob_w2_given_ABBBCDDD)
print("ABBBCDDD -> w1")

prob_w1_given_ABBBCDDD =  0.0001777567076295973
prob_w2_given_ABBBCDDD =  2.663726939368231e-05
ABBBCDDD -> w1


In [182]:
p_w1 = 0.5
p_w2 = 0.5

string2 = 'DADBCBAA'
prob_DADBCBAA_given_w1 = probability_of_a_sequence(string2, transition_w1, observation_w1, pie_w1)
prob_DADBCBAA_given_w2 = probability_of_a_sequence(string2, transition_w2, observation_w2, pie_w2)

prob_w1_given_DADBCBAA = prob_DADBCBAA_given_w1 * p_w1
prob_w2_given_DADBCBAA = prob_DADBCBAA_given_w2 * p_w2

print("prob_w1_given_DADBCBAA = ", prob_w1_given_DADBCBAA)
print("prob_w2_given_DADBCBAA = ", prob_w2_given_DADBCBAA)
print("DADBCBAA -> w2")

prob_w1_given_DADBCBAA =  1.1158941527173627e-15
prob_w2_given_DADBCBAA =  5.028040225429016e-06
DADBCBAA -> w2


In [183]:
p_w1 = 0.5
p_w2 = 0.5

string3 = 'CDCBABA' 
prob_CDCBABA_given_w1 = probability_of_a_sequence(string3, transition_w1, observation_w1, pie_w1)
prob_CDCBABA_given_w2 = probability_of_a_sequence(string3, transition_w2, observation_w2, pie_w2)

prob_w1_given_CDCBABA = prob_CDCBABA_given_w1 * p_w1
prob_w2_given_CDCBABA = prob_CDCBABA_given_w2 * p_w2

print("prob_w1_given_CDCBABA = ", prob_w1_given_CDCBABA)
print("prob_w1_given_CDCBABA = ", prob_w2_given_CDCBABA)
print("CDCBABA -> w2")

prob_w1_given_CDCBABA =  2.5652851084480972e-11
prob_w1_given_CDCBABA =  9.788943482592946e-05
CDCBABA -> w2


In [184]:
p_w1 = 0.5
p_w2 = 0.5

string4 = 'ADBBBCD'
prob_ADBBBCD_given_w1 = probability_of_a_sequence(string4, transition_w1, observation_w1, pie_w1)
prob_ADBBBCD_given_w2 = probability_of_a_sequence(string4, transition_w2, observation_w2, pie_w2)

prob_w1_given_ADBBBCD = prob_ADBBBCD_given_w1 * p_w1
prob_w2_given_ADBBBCD = prob_ADBBBCD_given_w2 * p_w2

print("prob_w1_given_ADBBBCD = ", prob_w1_given_ADBBBCD)
print("prob_w1_given_ADBBBCD = ", prob_w2_given_ADBBBCD)
print("ADBBBCD -> w1")

prob_w1_given_ADBBBCD =  9.843372802544132e-05
prob_w1_given_ADBBBCD =  1.6780406421376662e-12
ADBBBCD -> w1


In [185]:
####################### 3rd part #######################

#p_w1 + pw2 = 1
test_str = 'BADBDCBA'
prob_BADBDCBA_given_w1 = probability_of_a_sequence(test_str, transition_w1, observation_w1, pie_w1)
prob_BADBDCBA_given_w2 = probability_of_a_sequence(test_str, transition_w2, observation_w2, pie_w2)
ratio = prob_BADBDCBA_given_w2/prob_BADBDCBA_given_w1
x = np.array([[1, -1*ratio], [1,1]])
y = np.array([0, 1])
s = np.linalg.solve(x, y)
print("p(w1) = ",s[0])
print("p(w2) = ",s[1])

p(w1) =  5.94891580701867e-08
p(w2) =  0.999999940510842
